In [1]:
########################################################################
# import default libraries
########################################################################
import os
import sys
import gc
########################################################################


########################################################################
# import additional libraries
########################################################################
import numpy as np
import scipy.stats
# from import
from tqdm import tqdm
try:
    from sklearn.externals import joblib
except:
    import joblib
# original lib
import common as com
import keras_model
import pandas as pd
import keras
import librosa
import tensorflow as tf
import librosa.display
############
########################################################################
# load parameter.yaml
########################################################################
param = com.yaml_load()
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


In [2]:
def spec_augment_freq(spec: np.ndarray, num_mask=1, freq_masking_max_percentage=0.9):
    spec = spec.copy()
    spec[24:33, :] = -0
    spec[33:39, :] = -0
    return spec

In [3]:
target_dir = "/notebooks/dev_data/car1"
mode = True
dirs = com.select_dirs(param=param, mode=mode)
print(dirs)

# loop of the base dire/notebooks/model/model_car1.hdf5ctory
idx=0
print("\n===========================")
print("[{idx}/{total}] {target_dir}".format(target_dir=target_dir, idx=idx+1, total=len(dirs)))
machine_type = os.path.split(target_dir)[1]


def load_model(file_path):
    return keras.models.load_model(file_path, compile=False)

  
print("============== MODEL LOAD ==============")
# load model file
model_file = "{model}/model_{machine_type}.hdf5".format(model=param["model_directory"],
                                                        machine_type=machine_type)
print(model_file)
if not os.path.exists(model_file):
    com.logger.error("{} model not found ".format(machine_type))
    sys.exit(-1)
model = load_model(model_file)
model.summary()



2022-10-08 16:46:08,435 - INFO - load_directory <- development


['/notebooks/dev_data/car1', '/notebooks/dev_data/car8']

[1/2] /notebooks/dev_data/car1
============== MODEL LOAD ==============
/notebooks/model/model_car1.hdf5


2022-10-08 16:46:09.635883: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 128, 1  0           []                               
                                )]                                                                
                                                                                                  
 block_1_conv (Conv2D)          (None, 128, 64, 16)  416         ['input_1[0][0]']                
                                                                                                  
 block_1_conv_bn (BatchNormaliz  (None, 128, 64, 16)  64         ['block_1_conv[0][0]']           
 ation)                                                                                           
                                                                                              

In [4]:
print("\n============== BEGIN TEST FOR A SECTION ==============")
y_pred = [0. for k in range(20)]
y_pred_freq = [0. for k in range(20)]
y_pred_time = [0. for k in range(20)]
file_idx=1
# file_path='/content/drive/MyDrive/dev_data/car8/source_test/section_07_source_test_anomaly_0000 (126).wav'
file_path='dev_data/car1/source_test/section_00_source_test_anomaly_0000 (107).wav'
print(file_path)
# file_path='/content/drive/MyDrive/dev_data/car8/source_test/section_07_source_test_anomaly_0000 (7).wav'
# file_path='D:/B4/u-net/dev_data/car1/source_test/section_00_source_test_normal_0000 (1).wav'
vectors = com.file_to_vectors(file_path,
                              n_mels=128,
                              n_frames=1,
                              n_fft=1024,
                              hop_length=862,
                              power=2)
vectors = librosa.power_to_db(vectors)
vectors_masked = spec_augment_freq(vectors)
data = np.zeros((1, vectors.shape[1], vectors.shape[0]),float)
data_masked = np.zeros((1, vectors_masked.shape[1], vectors_masked.shape[0]), float)
data[0, :, :] = vectors.T
data_masked[0, :, :] = vectors_masked.T
data = data[:, :, :, np.newaxis]
data_masked = data_masked[:, :, :, np.newaxis]



print(data[0,:,:].shape)


============== BEGIN TEST FOR A SECTION ==============
dev_data/car1/source_test/section_00_source_test_anomaly_0000 (107).wav
(256, 128, 1)


In [5]:
import matplotlib.pyplot as plt

In [6]:
reconstruction = model.predict(data_masked)
y_pred = np.mean(np.square((data[0,:,24:39] - reconstruction[0,:,24:39])))
plt.gcf().set_size_inches(8, 8)
plt.subplot()
librosa.display.specshow(np.squeeze(data, axis=3)[0,:,:].T, cmap='magma')
plt.show()

plt.subplot()
plt.gcf().set_size_inches(8, 8)

librosa.display.specshow(np.squeeze(data_masked, axis=3)[0,:,:].T, cmap='magma')
plt.ylabel("Original")
plt.show()
plt.subplot()
plt.gcf().set_size_inches(8, 8)

librosa.display.specshow(np.squeeze(reconstruction, axis=3)[0,:,:].T[:128,:], cmap='magma')
plt.ylabel("Reconstruction")
# plt.subplot(3,1,3)
# y_ano = []
# y_nor = []
# x_ano = []
# x_nor = []
# n=-1
# for k in y_pred_time:
#     n+=1
#     if k > 13.3:
#         x_ano.append(n)
#         y_ano.append(1)
# n=-1
# for k in y_pred_time:
#     n+=1
#     if k < 13.3:
#         x_nor.append(n)
#         y_nor.append(0)
# x_ano = [x*5*3*512/16000 for x in x_ano]
# x_nor = [x*5*3*512/16000 for x in x_nor]
# x_nor[-1] = 10
# x_nor[0] = x_ano[-1]
# plt.plot(x_ano, y_ano, '-',color='r')
# plt.plot(x_nor, y_nor, '-',color='b')
# plt.fill_between(x_ano, y_ano,-2, alpha= 0.2, label='Anomaly', color='r')
# plt.ylim(0, 2)
# plt.xlim(0, 10)
# plt.xticks([])
# plt.yticks([])
# plt.legend(fontsize=8.5)
# plt.subplot(4,1,4)
# plt.plot(x, y_pred_time)
# plt.plot(x, y, '--',color='r')
# plt.ylim(0, max(y_pred_time)+10)
# plt.xlim(0, 10)
# plt.xlabel("Time (s)")
# plt.ylabel("Anomaly Score")
# plt.subplots_adjust(hspace=.0)
# plt.fill_between(x, y_pred_time, y, alpha= 0.2, where = (np.array(y_pred_time) > 12.5), label='Anomaly', color='r')
# plt.plot([], [], ' ', label="Mean Anomaly Score={:.2f}".format(anomaly_score_list[1][1]))
# plt.legend(fontsize=6.5)
plt.show()
print(y_pred)




2022-10-08 16:46:11.593319: E tensorflow/stream_executor/cuda/cuda_dnn.cc:373] Could not create cudnn handle: CUDNN_STATUS_INTERNAL_ERROR
2022-10-08 16:46:11.593581: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_ops.cc:1120 : UNIMPLEMENTED: DNN library is not found.


UnimplementedError: Graph execution error:

Detected at node 'model/block_1_conv/Conv2D' defined at (most recent call last):
    File "/usr/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.9/dist-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.9/dist-packages/traitlets/config/application.py", line 976, in launch_instance
      app.start()
    File "/usr/local/lib/python3.9/dist-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.9/dist-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.9/asyncio/base_events.py", line 601, in run_forever
      self._run_once()
    File "/usr/lib/python3.9/asyncio/base_event

In [7]:
!pip install tensorflow==2.8
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.6/497.6 MB 3.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 62.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 kB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 64.0 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.7.0
    Uninstalling keras-2.7.0:
      Successfully uninstalled keras-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0
    Uninstalling tensorflow-2.7.0:
      Successfully uninstalled tensorflow-2.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflic